In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
from bayesian_testing.experiments import PoissonDataTest

In [2]:
np.random.default_rng(52)

Generator(PCG64) at 0x1283A4BA0

In [3]:
# goals scored - more is better (duh...)
psg_goals_for = [5, 5, 7, 1, 3, 3, 1, 1, 2, 0, 1, 3, 4, 2, 5]
city_goals_for = [2, 4, 3, 4, 6, 1, 3, 6, 4, 0, 3, 1, 2, 1]
bayern_goals_for = [6, 2, 7, 1, 1, 2, 0, 4, 2, 5, 2, 6, 3, 6, 2]


# goals received - so less is better
psg_goals_against = [0, 2, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 3, 1, 0]
city_goals_against = [0, 0, 3, 2, 0, 1, 0, 3, 0, 1, 1, 0, 1, 2]
bayern_goals_against = [1, 0, 0, 1, 1, 2, 1, 0, 2, 0, 0, 2, 2, 1, 0]

In [4]:
# Poisson test for "goals for"
poisson_test_gf = PoissonDataTest()
poisson_test_gf.add_variant_data('psg', psg_goals_for)
# adding "city" with effective sample size 10 and the prior mean 2 (20/10):
poisson_test_gf.add_variant_data('city', city_goals_for, a_prior=20, b_prior=10)
# adding "bayern" with aggregated data instead of list of all observations
poisson_test_gf.add_variant_data_agg('bayern', totals=len(bayern_goals_for), sum_values=sum(bayern_goals_for))


# Poisson test for "goals against"
poisson_test_ga = PoissonDataTest()
poisson_test_ga.add_variant_data('psg', psg_goals_against)
poisson_test_ga.add_variant_data('city', city_goals_against)
poisson_test_ga.add_variant_data('bayern', bayern_goals_against)

In [5]:
# poisson_test_gf.probabs_of_being_best(sim_count = 20000, seed=52)
# poisson_test_gf.expected_loss(sim_count = 20000, seed=52)
results_gf = poisson_test_gf.evaluate()
print(pd.DataFrame(results_gf).to_markdown(tablefmt="grid", index=False))

+-----------+----------+--------------+--------------------+------------------+-------------------+-----------------+
| variant   |   totals |   sum_values |   observed_average |   posterior_mean |   prob_being_best |   expected_loss |
+===========+==========+==============+====================+==================+===================+=================+
| psg       |       15 |           43 |            2.86667 |          2.8543  |           0.24375 |        0.509445 |
+-----------+----------+--------------+--------------------+------------------+-------------------+-----------------+
| city      |       14 |           40 |            2.85714 |          2.5     |           0.05095 |        0.852689 |
+-----------+----------+--------------+--------------------+------------------+-------------------+-----------------+
| bayern    |       15 |           49 |            3.26667 |          3.25166 |           0.7053  |        0.111539 |
+-----------+----------+--------------+-----------------

In [6]:
# poisson_test_ga.probabs_of_being_best(sim_count = 20000, seed=52, min_is_best=True)
# poisson_test_ga.expected_loss(sim_count = 20000, seed=52, min_is_best=True)
results_ga = poisson_test_ga.evaluate(min_is_best=True)
print(pd.DataFrame(results_ga).to_markdown(tablefmt="grid", index=False))

+-----------+----------+--------------+--------------------+------------------+-------------------+-----------------+
| variant   |   totals |   sum_values |   observed_average |   posterior_mean |   prob_being_best |   expected_loss |
+===========+==========+==============+====================+==================+===================+=================+
| psg       |       15 |            9 |            0.6     |          0.60265 |           0.7599  |       0.0411113 |
+-----------+----------+--------------+--------------------+------------------+-------------------+-----------------+
| city      |       14 |           14 |            1       |          1       |           0.07465 |       0.439372  |
+-----------+----------+--------------+--------------------+------------------+-------------------+-----------------+
| bayern    |       15 |           13 |            0.86667 |          0.86755 |           0.16545 |       0.30863   |
+-----------+----------+--------------+-----------------